<a href="https://colab.research.google.com/github/daxzy-ai/SD-Anime-versions-webui/blob/main/N__FU_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Good anon, took the NAI files apart and put together a working web generator that works exactly like the original. 

I adapted it into a collab

**Colab was created by user: daswer123**

https://github.com/daswer123/stable-diffusion-colab

**If you liked the colab, give it a star :)**

##SETUP

In [ ]:
!pip install pyngrok

#Func to change model Path
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

def replace_line_in_file(file_name, line_to_search, text_to_replace):
    with open(file_name, 'r') as file:
        # read a list of lines into data
        data = file.readlines()

    for line in data:
        # if the line contains the string we're looking for,
        # write the line to the output file
        if line_to_search in line:
            replace_line(file_name, data.index(line), text_to_replace)


#UPDATE PYTHON TO 3.9.1
!python --version
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user
!python --version

# !pip install pyngrok

In [ ]:
# !pip install virtualenv
!python --version
!pip install uvicorn==0.18.3
!npm install -g localtunnel
!pip install gdown
!gdown https://huggingface.co/Daswer123/gfdsa/resolve/main/stable-diffusion1_4.zip -O naifu.zip #pls don't like it
!unzip naifu.zip
!mv models /content/program/
!rm -rf /content/naifu.zip #Remove unnecessary files

In [ ]:
#Versions
replace_line("/content/program/requirements.txt",1,"torch==1.12.1\n")
replace_line("/content/program/requirements.txt",2,"torchvision==0.13.1\n")
replace_line("/content/program/requirements.txt",3,"torchaudio==0.12.1\n")

replace_line("/content/program/requirements.txt",12,"pytorch-lightning==1.7.7\n")
replace_line("/content/program/requirements.txt",5,"fastapi==0.85.0\n")
replace_line("/content/program/requirements.txt",6,"uvicorn==0.18.3\n")
replace_line("/content/program/requirements.txt",7,"omegaconf==2.2.3\n")
replace_line("/content/program/requirements.txt",8,"transformers==4.23.1\n")
replace_line("/content/program/requirements.txt",16,"jsonmerge==1.8.0\n")

In [ ]:
%cd /content/program
!pip install -r /content/program/requirements.txt
!wget https://pastebin.com/raw/fBYfa6hJ -O run.sh

replace_line("run.sh", 7, 'export MODEL_PATH="models/animefull-final-pruned"\n')

sfw_dowload = False
custom_dowload = False
anything_dowload = False
anything_vae_dowload = False

In [ ]:
#Instal xformers
#Increases generation speed from 2.5it/s to 4it/s

from IPython.display import clear_output
import time
from IPython.display import HTML
from subprocess import getoutput
import os
s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

if (gpu=='T4'):
  %pip install -q https://github.com/daswer123/stable-diffusion-colab/raw/main/xformers%20prebuild/T4/python39/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl
  
elif (gpu=='P100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl

# #Need for xformers
# %cd /content/program
# !git clone https://github.com/openai/triton.git
# %cd triton/python
# !pip install -e .

## Select **full** or **sfw** model or you can upload your model

## you can reselect without reloading
The default setting is full

Select custom only when you load your own model

In [ ]:
Model = "full" #@param ["full","sfw","anything","custom"]
use_anything_vae = False #@param{type:"boolean"}

#@markdown if you want to upload click the checkbox and write the path to your model, after launching your model will be transferred to the folder custom, after you have uploaded you must uncheck it, and now when you select custom your **model** will be loaded
load_my_own_model = False #@param{type:"boolean"}
#@markdown #####Write the folder or theass as path to the folder, you can leave the field blank if the model is in the root of the drive
gdrive_path = "SD_models/waifu-diffusion1_3.ckpt" #@param {type:"string"}
#@markdown Your path will look like /content/drive/MyDrive/**gdrive_path**
load_model_via_hugginface = False #@param{type:"boolean"}
hugginface_url = "https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt" #@param {type:"string"}
#@markdown Insert a link like this

#@markdown https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt

if (load_my_own_model):
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/program/models
    !mkdir custom
    %cd custom

    print("moving model to custom folder")
    new_gdrive_path = "/content/drive/MyDrive/" + gdrive_path
    !cp $new_gdrive_path /content/program/models/custom/model.ckpt
    !gdown https://huggingface.co/Daswer123/gfdsa/raw/main/sfw.yaml -O config.yaml
    custom_dowload = True

if (load_model_via_hugginface):
  %cd /content/program/models
  !mkdir custom
  %cd custom

  print("dowload model and moving it to custom folder")
  !gdown $hugginface_url -O model.ckpt
  !gdown https://huggingface.co/Daswer123/gfdsa/raw/main/sfw.yaml -O config.yaml
  custom_dowload = True

if (Model == "sfw" and sfw_dowload == False):
  %cd /content/program/models
  !mkdir sfw
  %cd sfw 
  !gdown https://huggingface.co/Daswer123/gfdsa/resolve/main/sfw.ckpt -O model.ckpt
  !gdown https://huggingface.co/Daswer123/gfdsa/raw/main/sfw.yaml -O config.yaml
  sfw_dowload = True;

if (Model == "anything" and anything_dowload == False):
  %cd /content/program/models
  !mkdir anything
  %cd anything 
  !gdown https://huggingface.co/Daswer123/misc_models/resolve/main/Anything-V3.0-pruned-fp32.ckpt -O model.ckpt
  !gdown https://huggingface.co/Daswer123/gfdsa/raw/main/sfw.yaml -O config.yaml
  anything_dowload = True;

if (use_anything_vae == True and anything_vae_dowload == False):
  %cd /content/program/models
  !gdown https://huggingface.co/Daswer123/misc_models/resolve/main/Anything-V3.0.vae.pt -O /content/program/models/anything_vae.pt
  anything_vae_dowload = True

%cd /content/program

if (use_anything_vae == True):
  replace_line("run.sh", 13, 'export VAE_PATH="models/anything_vae.pt"\n')
else:
  replace_line("run.sh", 13, 'export VAE_PATH="models/animevae.pt"\n')

if (Model == "sfw"):
  replace_line("run.sh", 7, 'export MODEL_PATH="models/sfw"\n')
  print("Your model is: SFW")
elif (Model == "full"):
  replace_line("run.sh", 7, 'export MODEL_PATH="models/animefull-final-pruned"\n')
  print("Your model is: FULL")
elif (Model == "custom" and custom_dowload):
  replace_line("run.sh", 7, 'export MODEL_PATH="models/custom"\n')
  print("Your model is: CUSTOM")
elif (Model == "anything"):
  replace_line("run.sh", 7, 'export MODEL_PATH="models/anything"\n')
  print("Your model is: ANYTHING")
else:
  replace_line("run.sh", 7, 'export MODEL_PATH="models/animefull-final-pruned"\n')
  print("Your model must be loaded via gdrive before it can be run")
  print("Your model is: FULL")

## RUN 
Link to localtunel you can find in program folder in **lt.log**

In [ ]:
from pyngrok import ngrok
import time

%cd /content/program

#@markdown If Localtunnel has stopped working or you want to use ngrok,
use_ngrok = False #@param{type:"boolean"}
#@markdown Get <b>your</b> token for ngrok [here](https://dashboard.ngrok.com/get-started/your-authtoken) 
ngrok_token = "2HSKSXmR2Ip97Owdg3yIZLVdIk5_yayZbqeenwcgEuaxDP94" #@param {type:"string"}

if (use_ngrok):
  # # Get your authtoken from https://dashboard.ngrok.com/auth
  ngrok.set_auth_token(ngrok_token)
  # # Terminate open tunnels if exist
  ngrok.kill()
  # ngrok.disconnect(6969)
  time.sleep(3)
  public_url = ngrok.connect(6969).public_url
  
%cd /content/program
!nohup lt -l 0.0.0.0 -p 6969 > lt.log 2>&1 &  
import time
time.sleep(2)
with open('/content/program/lt.log', 'r') as testwritefile:
  print("\033[92m" + "Wait until code loads then follow this link")
  print(testwritefile.read())
  if (use_ngrok):
    print("Your ngrok link:")
    print(public_url)
  print("\033[95m")

!bash run.sh

# Notes
1) Keep in mind that in NAI Full "bad anatomy + low quality" and "low quality" is the same as "NSFW + bad anatomy + low quality" and "NSFW + low quality" 

But NAI Sfw uses the same options as the original generator

2) 99% of the time the result is identical to NAI Full and Sfw in their website

3) Once you have loaded the model with hugginface or gdrive you can, uncheck the checkbox and just switch the model without loading again

In [ ]:
#RESTART COLAB, IF YOU GET SOME TROUBLE
import os
os.kill(os.getpid(), 9)